In [1]:
import pandas as pd
import os
import shutil
import yaml
from pathlib import Path
import ast
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
from PIL import Image
from typing import List, Tuple, Dict
from tqdm.notebook import tqdm
import random
import json

In [2]:
emory = pd.read_csv('emory_final.csv')
vini = pd.read_csv('vinidir_final.csv')

In [3]:
emory = emory[~emory.category.isnull()].reset_index(drop=True)
emory = emory.rename(columns={'Image_Paths':'image_paths'})
emory['image_paths'] = emory['image_paths'].str.replace(r'^../', '', regex=True)
emory = emory.rename(columns={'asses': 'breast_birads', 'ViewPosition':'view','ImageLateralityFinal':'laterality'})
find_emory = emory[emory['num_roi']!=0].reset_index(drop=True)

In [4]:
vini = vini[vini.category!='No Finding'].reset_index(drop=True)
vini['image_paths'] = vini['image_paths'].str.replace(r'^../', '', regex=True)

In [5]:
import torch
import cv2
import numpy as np
from PIL import Image

def torch_CountUpContinuingOnes(b_arr):
    left = torch.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = torch.cummax(left, dim=-1)[0]

    rev_arr = torch.flip(b_arr, [-1])
    right = torch.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = torch.cummax(right, dim=-1)[0]
    right = len(rev_arr) - 1 - torch.flip(right, [-1])

    return right - left - 1

def torch_ExtractBreast_with_padding_single_side(img_ori, target_size=(512, 512), padding=1):
    # Detect background and set to zero
    img = torch.where(img_ori <= 20, torch.zeros_like(img_ori), img_ori)
    height, _ = img.shape

    # Extract the main breast region (same as before)
    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].to(torch.float32).std(dim=0) != 0
    continuing_ones = torch_CountUpContinuingOnes(b_arr)
    col_ind = torch.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]

    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].to(torch.float32).std(dim=1) != 0
    continuing_ones = torch_CountUpContinuingOnes(b_arr)
    row_ind = torch.where(continuing_ones == continuing_ones.max())[0]
    breast_region = img_ori[row_ind][:, col_ind]

    # Resize the extracted breast region while maintaining the aspect ratio
    breast_height, breast_width = breast_region.shape
    aspect_ratio = breast_width / breast_height

    # Define target dimensions based on aspect ratio
    if aspect_ratio > 1:
        # Wider than tall
        new_width = target_size[1] - padding
        new_height = int(new_width / aspect_ratio)
    else:
        # Taller than wide
        new_height = target_size[0] - padding
        new_width = int(new_height * aspect_ratio)

    resized_breast = cv2.resize(breast_region.cpu().numpy(), (new_width, new_height))
    resized_breast = torch.from_numpy(resized_breast)

    # Determine which side has lower intensity
    pad_x = target_size[1] - new_width
    pad_y = target_size[0] - new_height

    # Initialize offsets
    x_offset = 0
    y_offset = 0
    left_intensity = 0
    right_intensity = 0
    top_intensity = 0
    bottom_intensity = 0

    # Decide padding side for x-axis
    if pad_x > 0:
        left_intensity = resized_breast[:, 0].mean()
        right_intensity = resized_breast[:, -1].mean()
        if left_intensity < right_intensity:
            # Pad on the left side
            x_offset = pad_x
        else:
            # Pad on the right side
            x_offset = 0

    # Decide padding side for y-axis
    if pad_y > 0:
        top_intensity = resized_breast[0, :].mean()
        bottom_intensity = resized_breast[-1, :].mean()
        if top_intensity < bottom_intensity:
            # Pad on the top side
            y_offset = pad_y
        else:
            # Pad on the bottom side
            y_offset = 0

    # Create a padded image with the target size and place the resized breast region
    padded_img = torch.zeros(target_size, dtype=resized_breast.dtype)
    padded_img[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_breast
    
    data = {'padded_img':padded_img,
            'breast_region': breast_region,
            'left_intensity': left_intensity,
            'right_intensity': right_intensity,
            'top_intensity': top_intensity,
            'bottom_intensity': bottom_intensity,
            'target_size': target_size,
            'padding': padding
           }

    return data

# Apply the function in display_grid_transform for Emory dataset
def resize_with_padding_vini(img_np, target_size=(512, 512), padding=1):
    # Convert image to a torch tensor if not already
    img_torch = torch.from_numpy(img_np).to(torch.float32)

    # Ensure the breast region fits within a consistent scale
    breast_height, breast_width = img_torch.shape
    aspect_ratio = breast_width / breast_height

    # Define target dimensions based on aspect ratio
    if aspect_ratio > 1:
        # Wider than tall
        new_width = target_size[1] - padding
        new_height = int(new_width / aspect_ratio)
    else:
        # Taller than wide
        new_height = target_size[0] - padding
        new_width = int(new_height * aspect_ratio)
    
    # Resize while maintaining the aspect ratio
    resized_breast = cv2.resize(img_np, (new_width, new_height))
    resized_breast = torch.from_numpy(resized_breast)

     # Determine which side has lower intensity
    pad_x = target_size[1] - new_width
    pad_y = target_size[0] - new_height

    # Initialize offsets
    x_offset = 0
    y_offset = 0
    left_intensity = 0
    right_intensity = 0
    top_intensity = 0
    bottom_intensity = 0

    # Decide padding side for x-axis
    if pad_x > 0:
        left_intensity = resized_breast[:, 0].mean()
        right_intensity = resized_breast[:, -1].mean()
        if left_intensity < right_intensity:
            # Pad on the left side
            x_offset = pad_x
        else:
            # Pad on the right side
            x_offset = 0

    # Decide padding side for y-axis
    if pad_y > 0:
        top_intensity = resized_breast[0, :].mean()
        bottom_intensity = resized_breast[-1, :].mean()
        if top_intensity < bottom_intensity:
            # Pad on the top side
            y_offset = pad_y
        else:
            # Pad on the bottom side
            y_offset = 0

    # Create a padded image with the target size and place the resized breast region
    padded_img = torch.zeros(target_size, dtype=resized_breast.dtype)
    padded_img[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_breast

    data = {'padded_img':padded_img,
        'breast_region': img_torch,
        'left_intensity': left_intensity,
        'right_intensity': right_intensity,
        'top_intensity': top_intensity,
        'bottom_intensity': bottom_intensity,
        'target_size': target_size,
        'padding': padding
       }

    return data

In [6]:
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_img = clahe.apply(image)
    return equalized_img

def mean_variance_normalization(img_torch):
    img_min, img_max = img_torch.min(), img_torch.max()
    img_torch = (img_torch - img_min) / (img_max - img_min) * 255
    return img_torch

def transform_bbox(original_bbox, data):
    """
    Transform bounding box coordinates from original image space to resized and padded space.
    
    Args:
        original_bbox (tuple): (x1, y1, x2, y2) in original image space (3500x2680)
    Returns:
        tuple: Transformed (x1, y1, x2, y2) coordinates in the new image space
    """
    orig_x1, orig_y1, orig_x2, orig_y2 = original_bbox
    
    breast_region_shape = data['breast_region'].shape
    breast_height, breast_width = breast_region_shape
    target_size = data['target_size']
    padding = data['padding']
    left_intensity = data['left_intensity']
    right_intensity = data['right_intensity']
    top_intensity = data['top_intensity']
    bottom_intensity = data['bottom_intensity']
    
    # Calculate aspect ratio of the breast region
    aspect_ratio = breast_width / breast_height
    
    # Determine the dimensions after resize (before padding)
    if aspect_ratio > 1:
        # Wider than tall
        new_width = target_size[1] - padding
        new_height = int(new_width / aspect_ratio)
    else:
        # Taller than wide
        new_height = target_size[0] - padding
        new_width = int(new_height * aspect_ratio)
    
    # Calculate scaling factors
    scale_x = new_width / breast_width
    scale_y = new_height / breast_height
    
    # Scale the bbox coordinates
    scaled_x1 = int(orig_x1 * scale_x)
    scaled_y1 = int(orig_y1 * scale_y)
    scaled_x2 = int(orig_x2 * scale_x)
    scaled_y2 = int(orig_y2 * scale_y)
    
    # Calculate padding offsets based on image dimensions
    pad_x = target_size[1] - new_width
    pad_y = target_size[0] - new_height
    
    # Get the intensity values from the edges of the breast region
    # You'll need to pass these values or compute them here
    # For now, we'll assume they're computed the same way as in your original function
    x_offset = pad_x if left_intensity < right_intensity else 0
    y_offset = pad_y if top_intensity < bottom_intensity else 0
    
    # Apply offsets to the scaled coordinates
    final_x1 = scaled_x1 + x_offset
    final_y1 = scaled_y1 + y_offset
    final_x2 = scaled_x2 + x_offset
    final_y2 = scaled_y2 + y_offset
    
    # Ensure coordinates are within bounds
    final_x1 = max(0, min(final_x1, target_size[1]-1))
    final_y1 = max(0, min(final_y1, target_size[0]-1))
    final_x2 = max(0, min(final_x2, target_size[1]-1))
    final_y2 = max(0, min(final_y2, target_size[0]-1))
    
    return (final_x1, final_y1, final_x2, final_y2)

In [7]:
find_emory['category'].value_counts(), vini['category'].value_counts()

(category
 Asymmetry                   139
 Suspicious Calcification     80
 Mass                         31
 Architectural distortion     15
 Name: count, dtype: int64,
 category
 Mass                        1226
 Suspicious Calcification     453
 Focal Asymmetry              236
 Architectural Distortion      96
 Asymmetry                     92
 Suspicious Lymph Node         57
 Skin Thickening               49
 Global Asymmetry              24
 Nipple Retraction             14
 Skin Retraction                7
 Name: count, dtype: int64)

In [8]:
train_emory = find_emory.drop([12], axis=0).reset_index(drop=True)

In [9]:
train_emory = train_emory[train_emory['category']!='Architectural distortion'].reset_index()

In [10]:
cat_map = {'Asymmetry': 0, 'Suspicious Calcification': 1, 'Mass': 2}
train_emory['category'] = train_emory['category'].map(cat_map)

In [11]:
# Define the mapping dictionary
category_mapping = {
    'asymmetry': 0,
    'focal asymmetry':0,
    'global asymmetry':0,
    'architectural distortion':0,
    'nipple retraction':0,
    'skin retraction':0,
    'suspicious calcification':1,
    'mass':2,
    'suspicious lymph node':2,
    'skin thickening':2,
}

train_vini = vini.copy()
train_vini['category'] = train_vini['category'].str.lower()
train_vini['category'] = train_vini['category'].map(category_mapping)

In [12]:
from sklearn.model_selection import train_test_split

traine, vale = train_test_split(train_emory, test_size=0.1, stratify=train_emory['category'])
traine, vale = traine.reset_index(drop=True), vale.reset_index(drop=True)

trainv, valv = train_test_split(train_vini, test_size=0.1, stratify=train_vini['category'])
trainv, valv = trainv.reset_index(drop=True), valv.reset_index(drop=True)

In [13]:
os.makedirs('comb_yolo1/train/images', exist_ok=True)
os.makedirs('comb_yolo1/train/labels', exist_ok=True)

os.makedirs('comb_yolo1/val/images', exist_ok=True)
os.makedirs('comb_yolo1/val/labels', exist_ok=True)

In [14]:
def get_bboxes_yolo(data, dst_path, label_path, emory=True):
    for i, row in tqdm(enumerate(data.itertuples()), desc="Getting BBoxes.."):
        
        img_file_path = row.image_paths
        if os.path.exists(img_file_path):
            
            try:
                img_np = cv2.imread(img_file_path, cv2.IMREAD_GRAYSCALE)
                
                if emory:
                    img_torch = torch.from_numpy(img_np).to(torch.float32)
                    img_torch = mean_variance_normalization(img_torch)
                    image_data = torch_ExtractBreast_with_padding_single_side(img_torch, padding=0) 
                
                else:
                    img_np = mean_variance_normalization(img_np)
                    image_data = resize_with_padding_vini(img_np, padding=0)
                # Convert the result back to a NumPy array for visualization or further processing
                padded_img_np = image_data['padded_img'].cpu().numpy().astype(np.uint8)
    
                padded_img_rgb = np.repeat(padded_img_np[:, :, np.newaxis], 3, axis=2)
                padded_image = Image.fromarray(padded_img_rgb)
                filename = f"emory_{row.image_paths.split('/')[-1]}" if emory else f"vini_{row.image_paths.split('/')[-1]}"
                dst_img_path = os.path.join(dst_path, filename)
                padded_image.save(dst_img_path)
                
                labelname = f"emory_{row.image_paths.split('/')[-1].replace('.png','.txt')}" if emory else f"vini_{row.image_paths.split('/')[-1].replace('.png','.txt')}"
                label_file_path = os.path.join(label_path, labelname)
                width, height = padded_image.size
                
                if emory:
                    for roi in ast.literal_eval(row.ROI_coords):
                        y1, x1, y2, x2 = roi
                        # print(f"OLD ROI: x1:{x1}, y1:{y1}, x2:{x2}, y2:{y2}")
                        adj_roi=transform_bbox((x1, y1, x2, y2), image_data)
                        x1, y1, x2, y2 = adj_roi
                        # print(f"ADJ ROI: x1:{x1}, y1:{y1}, x2:{x2}, y2:{y2}")

                        x_center = ((x1 + x2) / 2) / width
                        y_center = ((y1 + y2) / 2) / height
                        wid = (x2 - x1) / width
                        hei = (y2 - y1) / height

                        text = [row.category, x_center, y_center, wid, hei]
                        with open(label_file_path, 'w') as f:
                            text = [str(i) for i in text]
                            f.write(' '.join(text))
                    
                else:
                    x1, y1, x2, y2 = row.resized_xmin, row.resized_ymin, row.resized_xmax, row.resized_ymax
                    adj_roi=transform_bbox((x1, y1, x2, y2), image_data)
                    x1, y1, x2, y2 = adj_roi

                    x_center = ((x1 + x2) / 2) / width
                    y_center = ((y1 + y2) / 2) / height
                    wid = (x2 - x1) / width
                    hei = (y2 - y1) / height

                    text = [row.category, x_center, y_center, wid, hei]
                    with open(label_file_path, 'w') as f:
                        text = [str(i) for i in text]
                        f.write(' '.join(text))
            except Exception as e:
                print(f"Cannot open File {img_file_path} because of exception{e}")
                continue
        else:
            print(f"FILE NOT FOUND {img_file_path}")

In [15]:
def get_bboxes_synth_yolo(class_data_dir, dst_path, label_path, ratio=1.0):
    """
    Processes synthetic images and saves a specified ratio of them along with their YOLO format labels.

    Args:
        class_data_dir (str): Path to the directory containing synthetic lesion classes.
        dst_path (str): Path to save processed images.
        label_path (str): Path to save YOLO labels.
        ratio (float): Ratio of images to process (0.0 to 1.0).
    """
    # Validate ratio
    if not (0.0 < ratio <= 1.0):
        raise ValueError("Ratio must be between 0.0 and 1.0")

    # Retrieve all synthetic images and corresponding ROIs across the 3 lesions
    lesion_dirs = {
        "Asymmetry": os.path.join(class_data_dir, "asymmetry"),
        "Suspicious Calcification": os.path.join(class_data_dir, "suspicious calcification"),
        "Mass": os.path.join(class_data_dir, "mass")
    }
    
    # Load images and ROIs
    image_roi_pairs = []
    for lesion_name, lesion_dir in lesion_dirs.items():
        roi_path = os.path.join(lesion_dir, "generated_rois.json")
        with open(roi_path, "r") as f:
            rois = json.load(f)

        lesion_images = sorted([os.path.join(lesion_dir, img) for img in os.listdir(lesion_dir) if img.endswith(".png")])
        if len(lesion_images) != len(rois):
            print(f"Warning: Number of images and ROIs do not match in {lesion_name}.")
            continue

        # Calculate the number of images to process based on the ratio
        num_images_to_process = int(len(lesion_images) * ratio)
        sampled_pairs = random.sample(list(zip(lesion_images, rois)), num_images_to_process)
        image_roi_pairs.extend([(img, roi, lesion_name) for img, roi in sampled_pairs])

    # Shuffle the images and corresponding ROIs
    random.shuffle(image_roi_pairs)

    # Process and copy images to the destination path
    for i, (img_path, roi, lesion_name) in tqdm(enumerate(image_roi_pairs), desc="Processing Images", unit="image"):
        try:
            # Build destination file paths
            image_id = f"{class_data_dir.split('_')[-1]}_{lesion_name.lower()}_{i:06d}"
            dst_img_path = os.path.join(dst_path, f"{image_id}.png")
            label_file_path = os.path.join(label_path, f"{image_id}.txt")

            # Open and process the image
            if os.path.exists(img_path):
                img = Image.open(img_path)
                shutil.copy(img_path, dst_img_path)
                width, height = img.size
                x1, y1, x2, y2 = roi

                # Convert bounding box to YOLO format
                x_center = ((x1 + x2) / 2) / width
                y_center = ((y1 + y2) / 2) / height
                bbox_width = (x2 - x1) / width
                bbox_height = (y2 - y1) / height

                # Write label file in YOLO format
                category = list(cat_map.keys()).index(lesion_name) # Get category index
                label_text = f"{category} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n"
                with open(label_file_path, 'w') as f:
                    f.write(label_text)
            else:
                print(f"Image file not found: {img_path}")

        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    print(f"Finished processing {len(image_roi_pairs)} images. Images and labels saved to {dst_path} and {label_path}.")


In [16]:
get_bboxes_yolo(traine, dst_path="comb_yolo1/train/images", label_path="comb_yolo1/train/labels")
get_bboxes_yolo(vale,  dst_path="comb_yolo1/val/images", label_path="comb_yolo1/val/labels")

get_bboxes_yolo(trainv, dst_path="comb_yolo1/train/images", label_path="comb_yolo1/train/labels", emory=False)
get_bboxes_yolo(valv,  dst_path="comb_yolo1/val/images", label_path="comb_yolo1/val/labels", emory=False)

Getting BBoxes..: 0it [00:00, ?it/s]

Cannot open File emory/images_png/16168291/1.2.845.113682.2750824972.1550469931.4661.2139.1/38180941880001850727199742295180123832.png because of exceptionexpected np.ndarray (got NoneType)


Getting BBoxes..: 0it [00:00, ?it/s]

Getting BBoxes..: 0it [00:00, ?it/s]

Getting BBoxes..: 0it [00:00, ?it/s]

In [17]:
print(len(os.listdir("comb_yolo1/train/images")), len(os.listdir("comb_yolo1/train/labels")))

1788 1788


In [18]:
get_bboxes_synth_yolo(class_data_dir="synthetic_lesions",dst_path="comb_yolo1/train/images",label_path="comb_yolo1/train/labels", ratio=0.375)
get_bboxes_synth_yolo(class_data_dir="synthetic_vlesions",dst_path="comb_yolo1/train/images",label_path="comb_yolo1/train/labels", ratio=0.375)

Processing Images: 0image [00:00, ?image/s]

Finished processing 2250 images. Images and labels saved to comb_yolo1/train/images and comb_yolo1/train/labels.


Processing Images: 0image [00:00, ?image/s]

Finished processing 2250 images. Images and labels saved to comb_yolo1/train/images and comb_yolo1/train/labels.


In [19]:
print(len(os.listdir("comb_yolo1/train/images")), len(os.listdir("comb_yolo1/train/labels")))

6288 6288


In [20]:
data = dict(
    train = "/notebooks/comb_yolo1/train/images",
    val = "/notebooks/comb_yolo1/val/images",

    nc = len(cat_map),
    names = list(cat_map.keys())
)

with open('comb_yolo1/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile)

In [21]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt')

In [ ]:
yaml_file = 'comb_yolo1/data.yaml'

model.train(data=yaml_file,
            epochs=50,
            patience=20,
            batch=32,
            optimizer='Adam',
            lr0 = 1e-4,
            lrf = 1e-3,
            weight_decay = 5e-4,
            name = f'yolov8s_3classes_comb2_0.5',
            save=True,
            amp=True,
            val=True)

In [ ]:
# Define the path to the directory
post_training_files_path = 'runs/detect/yolov8s_3classes_comb2_0.5'

# Construct the path to the best model weights file using os.path.join
best_model_path = os.path.join(post_training_files_path, 'weights/best.pt')

# Load the best model weights into the YOLO model
best_model = YOLO(best_model_path)

# Validate the best model using the validation set with default parameters
metrics = best_model.val(split='val')

In [ ]:
# Convert the dictionary to a pandas DataFrame and use the keys as the index
metrics = pd.DataFrame.from_dict(metrics.results_dict, orient='index', columns=['Metric Value'])

In [ ]:
# Configure the visual appearance of Seaborn plots
sns.set(rc={'axes.facecolor': '#9b63b8'}, style='darkgrid')

def display_images(post_training_files_path, image_files):
    
    for image_file in image_files:
        image_path = os.path.join(post_training_files_path, image_file)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 10), dpi=120)
        plt.imshow(img)
        plt.axis('off')
        plt.show()

# List of image files to display
image_files = [
    'confusion_matrix_normalized.png',
    'F1_curve.png',
    'P_curve.png',
    'R_curve.png',
    'PR_curve.png',
    'results.png'
]

# Display the images
display_images(post_training_files_path, image_files)